# MonteCarlo Simulation
In this notebook we cover a full example using MonteCarlo Simulation.

In [ ]:
# install all the azure kusto modules
%pip install azure.kusto.data
%pip install azure-kusto-ingest
%pip install azure.identity
%pip install pandas

In [ ]:
# install all the azure kusto modules
%pip install pyodbc

                                              0.0/68.7 kB ? eta -:--:--
     -----                                    10.2/68.7 kB ? eta -:--:--
     ----------------                       30.7/68.7 kB 217.9 kB/s eta 0:00:01
     ---------------------------------      61.4/68.7 kB 363.1 kB/s eta 0:00:01
     -------------------------------------- 68.7/68.7 kB 372.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd
import json

class KustoConnection():
    def __init__(self, configuration : dict):
        uri = configuration.get('cluster_uri')
        if (not str(uri).startswith('https://')):
            uri = "https://" + uri
        if (not str(uri).endswith('.kusto.windows.net')) :
            uri = uri + '.kusto.windows.net'
        self.cluster_uri = uri
        self.database = configuration.get('database')
        cred = ChainedTokenCredential(ManagedIdentityCredential(), AzureCliCredential(), DefaultAzureCredential())
        connection_string = KustoConnectionStringBuilder.with_aad_application_token_authentication(self.cluster_uri, 
                                                                                                   cred.get_token(self.cluster_uri + '/.default').token)        
        # Create a KustoClient instance
        self.client = KustoClient(connection_string)

    def query(self, query: str, properties: ClientRequestProperties = None) -> pd.DataFrame:
        """
        Execute a Kusto query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        response = self.client.execute_query(self.database, query, properties)
        df = dataframe_from_result_table(response.primary_results[0])
        return df

    def get_schema(self, table_name: str) -> dict:
        """
        Get a table schema
        @param table_name: name of table
        @returns a dictionary mapping column names to kusto types
        """
        schema = self.client.execute_mgmt(self.database, f".show table {table_name} schema as json").primary_results[0][0]['Schema']
        return json.loads(schema)

    def close(self):
        """
        Close the Kusto client.
        """
        self.client.close()

from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
import struct
import pyodbc
import pandas as pd 

class DatabaseConnection():
    def __init__(self, configuration : dict):
        server = configuration.get('server')
        if (not str(server).endswith('.database.windows.net')) :
            server = server + '.database.windows.net';
        self.sever = server;
        self.database = configuration.get('database');

        self.server = 'eppmreportingdev.database.windows.net';
        self.database = 'DCEPDev';
        query = 'SELECT TOP 10 * from [MonteCarlo].[SimulationResult];'
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        databaseToken = credential.get_token('https://database.windows.net/');
        tokenb = bytes(databaseToken[0], "UTF-8");
        exptoken = b'';
        for i in tokenb:
            exptoken += bytes({i});
            exptoken += bytes(1);
        tokenstruct = struct.pack("=i", len(exptoken)) + exptoken;
        connString = "Driver={ODBC Driver 17 for SQL Server};SERVER="+self.sever+";DATABASE="+self.database+"";
        SQL_COPT_SS_ACCESS_TOKEN = 1256; 
        self.connection = pyodbc.connect(connString, attrs_before = {SQL_COPT_SS_ACCESS_TOKEN:tokenstruct});


    # write a database to database, mode = append or replace
    def insert(self, df : pd.DataFrame, table :str):
        """
        Execute a SQL write.
        @param df: Data Frame.        
        @param table: Table Name.
        """
        statement = '''
        INSERT INTO {table}
        (
            {columns}
        )
        VALUES({place_holders})
        '''
        # add protection on column name
        columns = df.columns.tolist();
        for i in range(len(columns)):
            columns[i] = '[' + columns[i] + ']';
        statement = statement.format(table = table, 
                                     columns = (',').join(columns), 
                                     place_holders = (',').join(['?'] * len(df.columns)));
        print(statement);
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.executemany(statement, df.values.tolist());
        cursor.commit();
        cursor.close();

    # execute sql command
    def execute(self, statement : str):
        """
        Execute a SQL command.
        @param query: query.        
        """
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.execute(statement);
        cursor.commit();
        cursor.close();


    def query(self, query: str) -> pd.DataFrame:
        """
        Execute a SQL query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        # Create a Pandas dataframe from the results
        
        df = pd.read_sql(query, self.connection);

        # Print the dataframe
        print(df);
        return df

    def close(self):
        """
        Close the Database client.
        """
        self.connection.close();

class DCEPKustoData():
    def __init__(self):
        self.kusto_client = KustoConnection({'cluster_uri' : "dpxedwhprod.southcentralus", 'database': "Dpxedwhprod"});
        self.query = {
        'IFP_CS_FS_IST_CR_Tranche_List' : 
        '''
        DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter
        (
            DCEPAllMilestonesCurrent 
            | summarize by DCShortCode, ProjectName, Tranche, 
                Colo_Ready_L2Milestone = "Colo Ready", 
                ProjectType, 
                CR_TaskStartDate = ColoReadyDate, 
                CR_TaskFinishDate = ColoReadyDate 
        )
        on $left.ProjectName == $right.ProjectName and $left.Tranche == $right.Tranche
        | project-away ProjectName1, Tranche1        
        | join kind = leftouter
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName, Tranche, 
                Design_IFP_Milestone = "Design IFP", 
                ProjectType, 
                IFP_TaskStartDate = DesignIFP, 
                IFP_TaskFinishDate = DesignIFP        
        )
        on $left.ProjectName == $right.ProjectName
        | project-away DCShortCode1, ProjectName1, ProjectType1, Tranche1          
        | join kind = leftouter
        (
            DCEPAllMilestonesCurrent 
              | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
              | summarize by DCShortCode, ProjectName,
                    Tranche, CS_L2Milestone = "Construction Start", 
                    ProjectType, CS_TaskStartDate = ConstructionDate, 
                    CS_TaskFinishDate = ConstructionDate            
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away DCShortCode1, ProjectName1, ProjectType1, Tranche1  
        | join kind = leftouter
        (
            EstimatedFoundationStart
            | summarize by 
                DCShortCode, ProjectName, Tranche = tostring(Tranche), 
                FS_L2Milestone="Foundation", ProjectType, 
                FS_TaskStartDate = Foudation_TaskStartDate, 
                FS_TaskFinishDate = Foudation_TaskFinishDate
        )
        on $left.ProjectName == $right.ProjectName and $left.Tranche == $right.Tranche
        | project-away DCShortCode1, ProjectName1, ProjectType1, Tranche1 
        | join kind = leftouter
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName, Tranche, 
                  IST_L2Milestone = "IST Complete", ProjectType, 
                  IST_TaskStartDate = ISTDate, 
                  IST_TaskFinishDate = ISTDate
        )
        on $left.ProjectName == $right.ProjectName and $left.Tranche == $right.Tranche
        | project-away DCShortCode1, ProjectName1, ProjectType1, Tranche1  
        | extend IST_2_CR = case
          (
              IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d) 
                and FS_TaskStartDate <= ago(1d) and IST_TaskStartDate <= ago(1d) 
                and CR_TaskStartDate > ago(1d),  1, 
                0
          ),
          FS_2_CR = case
          (
              IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d)  
                and FS_TaskStartDate <= ago(1d) and IST_TaskStartDate > ago(1d)  
                and CR_TaskStartDate > ago(1d),  1, 
                0
          ),
          CS_2_CR = case
          (
              IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d)  
                and FS_TaskStartDate  > ago(1d) and IST_TaskStartDate > ago(1d)  
                and CR_TaskStartDate > ago(1d),  1, 
                0
          ),
          IFP_2_CR = case
          (
              IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  > ago(1d) 
                and FS_TaskStartDate  > ago(1d) and IST_TaskStartDate > ago(1d) 
                and CR_TaskStartDate > ago(1d),  1, 
                0
          )
        | where IFP_2_CR + CS_2_CR + FS_2_CR + IST_2_CR == 1
        ''',
        
        'IFP_CS_FS_IST_CR_Duration' : 
        '''
        let CRFullSet =  
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by 
                  DCShortCode, ProjectName, ProjectType, Tranche, L2Milestone = "Colo Ready", 
                  CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        let ISTFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by 
                  DCShortCode, ProjectName, ProjectType, Tranche, L2Milestone = "IST Complete",  
                  IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let FSFullSet = 
        (
          EstimatedFoundationStart
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by 
                DCShortCode, ProjectName, ProjectType, Tranche = tostring(Tranche), 
                L2Milestone="Foundation", 
                FS_TaskStartDate = Foudation_TaskStartDate, 
                FS_TaskFinishDate = Foudation_TaskFinishDate
        );
        let CSFullSet = 
        (
          DCEPAllMilestonesCurrent
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by ProjectName, Tranche
          | join kind = leftouter  
          (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build")
            | summarize by 
                DCShortCode, ProjectName,Tranche, L2Milestone = "Construction Start", 
                ProjectType, CS_TaskStartDate = ConstructionDate, 
                CS_TaskFinishDate = ConstructionDate
          ) 
          on $left.ProjectName == $right.ProjectName 
          | project-away ProjectName1, Tranche1
          | where isnotempty(CS_TaskFinishDate)
        );
        let IFPFullSet = 
        (
          DCEPAllMilestonesCurrent
          | where ProjectType in ("Build","Build PH","OAI Build") 
          | summarize by ProjectName, Tranche
          | join kind = leftouter  
          (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, L2Milestone = "Design IFP", ProjectType, 
                           IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
          )
          on $left.ProjectName == $right.ProjectName 
          | project-away ProjectName1, Tranche1
        );         
        CRFullSet        
        | join kind= leftouter ISTFullSet 
        on $left.ProjectName == $right.ProjectName and $left.Tranche == $right.Tranche 
        | extend ist_cr_duration = datetime_diff("day",CR_TaskFinishDate, IST_TaskFinishDate)
        | project-away DCShortCode1, ProjectType1, ProjectName1, Tranche1, L2Milestone1
        | join kind= leftouter (FSFullSet) 
        on $left.ProjectName == $right.ProjectName and $left.Tranche == $right.Tranche
        | extend fs_cr_duration = datetime_diff("day", CR_TaskFinishDate, FS_TaskStartDate)
        | project-away DCShortCode1, ProjectType1, ProjectName1, Tranche1, L2Milestone1
        | join kind= leftouter (CSFullSet) 
        on $left.ProjectName == $right.ProjectName and $left.Tranche == $right.Tranche
        | extend cs_cr_duration = datetime_diff("day", CR_TaskFinishDate, CS_TaskFinishDate)
        | project-away DCShortCode1, ProjectType1, ProjectName1, Tranche1, L2Milestone1
        | join kind = inner (IFPFullSet) 
        on $left.ProjectName == $right.ProjectName and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche 
        | extend ifp_cr_duration = datetime_diff("day", CR_TaskFinishDate, IFP_TaskFinishDate)
        | project-away DCShortCode1, ProjectType1, ProjectName1, Tranche1, L2Milestone1
        | order by DCShortCode asc, ProjectType, Tranche asc        
        ''',
        
        'IFP_CS_FS_IST_CR_p5080_min_max' : 
        '''
        let FY = 
          iff(
              monthofyear(ago(0d)) > 6, 
              datetime_part("year", ago(0d)) + 1, 
              datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete"  
        and ProjectType in ("Build","Build PH","OAI Build") 
        and ColoReadyDate <= todatetime(FY_Enddate) 
        and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize min(ColoReadyDate) 
          by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate;         
        let IFPFullSet =  DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region, ProjectType, 
              IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where isnotempty(IFP_TaskStartDate);
        let IFP_CR_Duration = IFPFullSet 
        | join kind = inner CRFullSet on $left.DCShortCode == $right.DCShortCode 
          and $left.ProjectType == $right.ProjectType  and $left.Tranche == $right.Tranche 
        | project DCShortCode, ProjectType, Tranche, IFP_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate)
        | extend leadtime = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate)*1.0 
        | order by DCShortCode, ProjectType, Tranche asc        
        | extend ID="IFPCR"
        | where duration  > 0;       
        let IFP_CR_Outlier = IFP_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) by Tranche
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="IFPCR"; 
        let IFP_CR = IFP_CR_Duration 
        | join kind = leftouter IFP_CR_Outlier on $left.ID == $right.ID and $left.Tranche == $right.Tranche
        | extend Outlier = 
        iff(leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier") 
        | where Outlier != "Outlier"; 
        let CSFullSet = DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter 
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by 
                  DCShortCode, ProjectName,Tranche, Region,
                  L2Milestone = "Construction Start", ProjectType, 
                  CS_TaskStartDate = ConstructionDate, 
                  CS_TaskFinishDate = ConstructionDate
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where isnotempty(CS_TaskStartDate)
        | summarize CS_TaskStartDate = minif(CS_TaskStartDate, 
           L2Milestone in ("EarthWorks", "Construction Start")) by 
           DCShortCode, ProjectType, ProjectName, Tranche, Region;
        let CS_CR_Duration = CSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche
        | project DCShortCode, ProjectType, Tranche, CS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, CS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, CS_TaskStartDate)*1.0 
        | order by DCShortCode asc | extend ID="CSCR"| where duration  > 0; 
        let CS_CR_Outlier = CS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75), P25 = percentiles(leadtime,25) by Tranche
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="CSCR"; 
        let CS_CR = CS_CR_Duration 
        | join kind = leftouter CS_CR_Outlier on $left.ID == $right.ID and $left.Tranche == $right.Tranche
        | extend Outlier = iff
        (
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
        ) 
        | where Outlier != "Outlier";                
        let FSFullSet =
        EstimatedFoundationStart
        | where ProjectType in ("Build","Build PH","OAI Build")
        | summarize by DCShortCode, ProjectType, ProjectName, 
          Tranche = tostring(Tranche),Region,FS_TaskStartDate =Foudation_TaskStartDate;        
        let FS_CR_Duration = 
        FSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche 
        | project DCShortCode, Tranche, FS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, FS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, FS_TaskStartDate)*1.0 
        | order by DCShortCode, Tranche asc 
        | extend ID="FSCR"
        | where duration  > 0;        
        let FS_CR_Outlier = FS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) by Tranche
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="FSCR"; 
        let FS_CR = 
        FS_CR_Duration 
        | join kind = leftouter FS_CR_Outlier on $left.ID == $right.ID and $left.Tranche == $right.Tranche
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier";        
        let ISTFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize min(ColoReadyDate) by 
          DCShortCode, ProjectType, Tranche, ProjectName, Region, ISTDate; 
        let IST_CR_Duration = ISTFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType and $left.Tranche == $right.Tranche
        | project DCShortCode, ProjectType, Tranche, ISTDate, ColoReadyDate, 
        duration = datetime_diff("day",ColoReadyDate, ISTDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, ISTDate)*1.0 
        | order by DCShortCode asc | extend ID="ISTCR"
        | where duration  > 0; 
        let IST_CR_Outlier = IST_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) by Tranche
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="ISTCR"; 
        let IST_CR = IST_CR_Duration 
        | join kind = leftouter IST_CR_Outlier on $left.ID == $right.ID and $left.Tranche == $right.Tranche
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier";      
        let IFP_CR_p5080_min_max = IFP_CR 
        | summarize 
            IFP_CR_percentile_duration_50 = percentiles(duration, 50), 
            IFP_CR_percentile_duration_80 = percentiles(duration, 80), 
            IFP_CR_duration_min = min(duration), 
            IFP_CR_duration_max = max(duration) 
            by Tranche;
        let CS_CR_p5080_min_max = CS_CR 
        | summarize 
            CS_CR_percentile_duration_50 = percentiles(duration, 50), 
            CS_CR_percentile_duration_80 = percentiles(duration, 80), 
            CS_CR_duration_min = min(duration), 
            CS_CR_duration_max = max(duration) 
            by Tranche;
        let FS_CR_p5080_min_max = FS_CR 
        | summarize 
            FS_CR_percentile_duration_50 = percentiles(duration, 50), 
            FS_CR_percentile_duration_80 = percentiles(duration, 80), 
            FS_CR_duration_min = min(duration), 
            FS_CR_duration_max = max(duration) 
            by Tranche;
        let IST_CR_p5080_min_max = IST_CR 
        | summarize 
            IST_CR_percentile_duration_50 = percentiles(duration, 50), 
            IST_CR_percentile_duration_80 = percentiles(duration, 80), 
            IST_CR_duration_min = min(duration), 
            IST_CR_duration_max = max(duration) 
            by Tranche;
        IFP_CR_p5080_min_max
        | join kind = leftouter CS_CR_p5080_min_max
        on $left.Tranche == $right.Tranche
        | join kind = leftouter FS_CR_p5080_min_max
        on $left.Tranche == $right.Tranche
        | join kind = leftouter IST_CR_p5080_min_max
        on $left.Tranche == $right.Tranche
        | order by Tranche asc
         | project-away Tranche1, Tranche2, Tranche3;
        '''
        };
    
    def get_duration_query(self):
        return self.duration;

    def query_kusto(self, query): 
        response = self.kusto_client.query(query);
        result = response.reset_index();         
        return result;

    def query_static_data(self):
        self.data = {};
        query = self.query['IFP_CS_FS_IST_CR_Tranche_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['IFP_CS_FS_IST_CR_Tranche_List'] = result;

        query = self.query['IFP_CS_FS_IST_CR_Duration'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['IFP_CS_FS_IST_CR_Duration'] = result;

        query = self.query['IFP_CS_FS_IST_CR_p5080_min_max'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['IFP_CS_FS_IST_CR_p5080_min_max'] = result;

        return self.data;

from asyncio.windows_events import NULL
import numpy as np
import os
import random
import pandas as pd
from time import time;

databaseConn = DatabaseConnection({'server' : 'eppmreportingdev.database.windows.net', 'database' :'DCEPDev'});
databaseConn.execute('DELETE FROM [MonteCarlo].[SimulationResult_HM]')
client = DCEPKustoData();
kusto_data = client.query_static_data();
N = 100000
stages = 1

project_tranches = kusto_data['IFP_CS_FS_IST_CR_Tranche_List'];
#print(project_tranches);

durations = kusto_data['IFP_CS_FS_IST_CR_Duration'];
#print(duration);

p5080_min_max = kusto_data['IFP_CS_FS_IST_CR_p5080_min_max'];
#print(p5080_min_max);

for index, project_tranche in project_tranches.iterrows():
    print('Project Tranche = ');
    print(project_tranche);
    print('p5080_min_max = ');
    print(p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]);
    print('Duration = ');
    print(durations[(durations['ProjectName'] == project_tranche['ProjectName']) & (durations['Tranche'] == project_tranche['Tranche'])]);
    stages = 1;
    T = np.empty(shape=(N, stages + 1));
    TaskTimes=[];
    TotalTime=[];
    Lh=[];
    duration = 0
    result = pd.DataFrame
    tasktime = []; 
    schedule_type = '';

    if project_tranche['IFP_2_CR'] == 1 :
        p50 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IFP_CR_percentile_duration_50'].values[0];
        p80 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IFP_CR_percentile_duration_80'].values[0];
        min = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IFP_CR_duration_min'].values[0];
        max = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IFP_CR_duration_max'].values[0];
        duration = durations[(durations['ProjectName'] == project_tranche['ProjectName']) & (durations['Tranche'] == project_tranche['Tranche'])]['ifp_cr_duration'].values[0];
        schedule_type = 'IFP_2_CR';
    elif project_tranche['CS_2_CR'] == 1 :
        p50 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['CS_CR_percentile_duration_50'].values[0];
        p80 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['CS_CR_percentile_duration_80'].values[0];
        min = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['CS_CR_duration_min'].values[0];
        max = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['CS_CR_duration_max'].values[0];
        duration = durations[(durations['ProjectName'] == project_tranche['ProjectName']) & (durations['Tranche'] == project_tranche['Tranche'])]['cs_cr_duration'].values[0];
        schedule_type = 'CS_2_CR';
    elif project_tranche['FS_2_CR'] == 1 :
        p50 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['FS_CR_percentile_duration_50'].values[0];
        p80 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['FS_CR_percentile_duration_80'].values[0];
        min = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['FS_CR_duration_min'].values[0];
        max = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['FS_CR_duration_max'].values[0];
        duration = durations[(durations['ProjectName'] == project_tranche['ProjectName']) & (durations['Tranche'] == project_tranche['Tranche'])]['fs_cr_duration'].values[0];
        schedule_type = 'FS_2_CR';
    elif project_tranche['IST_2_CR'] == 1 :
        p50 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IST_CR_percentile_duration_50'].values[0];
        p80 = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IST_CR_percentile_duration_80'].values[0];
        min = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IST_CR_duration_min'].values[0];
        max = p5080_min_max[p5080_min_max['Tranche'] ==  project_tranche['Tranche']]['IST_CR_duration_max'].values[0];
        duration = durations[(durations['ProjectName'] == project_tranche['ProjectName']) & (durations['Tranche'] == project_tranche['Tranche'])]['ist_cr_duration'].values[0];
        schedule_type = 'IST_2_CR';
    
    tasktime.append(min);
    tasktime.append(p50);
    tasktime.append(max);
    TaskTimes.append(tasktime);
    t0 = time();
    print('Time of start loop ', t0);
    for i in range(stages):
      Lh.append((TaskTimes[i][1]-TaskTimes[i][0]) / (TaskTimes[i][2]-TaskTimes[i][0]));

        #Lh running time result
    dt = time()-t0;
    print('Time to run Lh '+ 'dt_loop=%s' % round(dt, 10));

    t2 = time();
    print('Time of start CS to CR loop ', t2);
    for p in range(N):
      for i in range(stages):
        trand=random.random()
        if Lh[i] is not pd.NA:
          if (trand < Lh[i]):
            T[p][i] = TaskTimes[i][0] + np.sqrt(trand*(TaskTimes[i][1]-  TaskTimes[i][0])*(TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = duration;
          else:
            T[p][i] = TaskTimes[i][2] - np.sqrt((1-trand)*(TaskTimes[i][2] - TaskTimes[i][1]) * (TaskTimes[i][2]-TaskTimes[i][0]));
            T[p][i+1] = duration;
      TotalTime.append(T[p][0]);
    dt2_loop = time()-t2
    print('Time to run simulation loop '+ 'dt2_loop=%s' % round(dt2_loop, 10))

    Data = pd.DataFrame(T,columns=["MC_Simulation", "DCEP_Duration"]);
    pd.set_option("display.max_columns", None);
    df = Data.describe(percentiles=[.25, .50, .75,.95]).reset_index();
    df.rename(columns={"index": "statistics"}, inplace=True)
    df.insert(0,'ProjectName', [project_tranche["ProjectName"]] *len(df));
    df.insert(1,'Tranche', [project_tranche["Tranche"]] *len(df));
    df.insert(2,'ScheduleType', [schedule_type] *len(df));
    print(df);

    databaseConn.insert(df, table = '[MonteCarlo].[SimulationResult_HM]');

    #run simulation loop running time result
    dt_list = time()-t0
    print("TotalTime Size = " + str(len(TotalTime)));
    print('Project = '+ project_tranche["ProjectName"] + ', Tranche = '+ project_tranche["Tranche"]  + ', Time to run Bucket once '+ 'dt_loop=%s' % round(dt_list, 10));
    print('IFP_2_CR = '+ str(project_tranche["IFP_2_CR"]) + ', CS_2_CR = ' + str(project_tranche["CS_2_CR"]) + ', FS_2_CR = ' + str(project_tranche["FS_2_CR"]) + ', IST_2_CR = '+ str(project_tranche["IST_2_CR"]));
 


    


Project Tranche = 
index                                             0
ProjectName                                   LON01
Tranche                                           1
DCShortCode                                   LON01
Colo_Ready_L2Milestone                   Colo Ready
ProjectType                                   Build
CR_TaskStartDate          2025-06-24 00:00:00+00:00
CR_TaskFinishDate         2025-06-24 00:00:00+00:00
Design_IFP_Milestone                     Design IFP
IFP_TaskStartDate         2022-02-18 00:00:00+00:00
IFP_TaskFinishDate        2022-02-18 00:00:00+00:00
CS_L2Milestone                   Construction Start
CS_TaskStartDate          2023-01-31 00:00:00+00:00
CS_TaskFinishDate         2023-01-31 00:00:00+00:00
FS_L2Milestone                           Foundation
FS_TaskStartDate          2023-02-21 08:00:00+00:00
FS_TaskFinishDate         2024-02-23 17:00:00+00:00
IST_L2Milestone                        IST Complete
IST_TaskStartDate         2025-02-20 00:00:00